In [ ]:
import os
import urllib.request
import zipfile
import tarfile

import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import numpy as np

In [ ]:
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [ ]:
import sklearn
print(sklearn.__version__)

In [ ]:
# pip install -U scikit-learn

In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, data_home="./data/", as_frame=False)  

In [ ]:
X = mnist.data
y = mnist.target

In [ ]:
plt.imshow(X[0].reshape(28, 28), cmap='gray')
print("この画像データのラベルは{}です".format(y[0]))

In [ ]:
data_dir_path = "./data/img_78/"
if not os.path.exists(data_dir_path):
    os.mkdir(data_dir_path)

In [ ]:
count7=0
count8=0
max_num=200

for i in range(len(X)):
    
    if (y[i] is "7") and (count7<max_num):
        file_path="./data/img_78/img_7_"+str(count7)+".jpg"
        im_f=(X[i].reshape(28, 28))
        pil_img_f = Image.fromarray(im_f.astype(np.uint8)) 
        pil_img_f = pil_img_f.resize((64, 64), Image.BICUBIC) 
        pil_img_f.save(file_path)
        count7+=1 
    
    if (y[i] is "8") and (count8<max_num):
        file_path="./data/img_78/img_8_"+str(count8)+".jpg"
        im_f=(X[i].reshape(28, 28))
        pil_img_f = Image.fromarray(im_f.astype(np.uint8)) 
        pil_img_f = pil_img_f.resize((64, 64), Image.BICUBIC) 
        pil_img_f.save(file_path) 
        count8+=1 

In [ ]:
# 패키지 import
import random
import math
import time
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import transforms

In [ ]:
# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [ ]:
# input = torch.tensor([[[[1., 1.], [2., 2.]]]])
# print("入力データ")
# print(input)
# print("-----")

# print("通常の畳み込み")
# m = nn.Conv2d(1, 1, 2, stride=1, bias=False)
# m.weight.data[0, 0, 0, 0] = 1
# m.weight.data[0, 0, 0, 1] = 2
# m.weight.data[0, 0, 1, 0] = 3
# m.weight.data[0, 0, 1, 1] = 4
# print("カーネル")
# print(m.weight)
# print("出力")
# print(m(input))

# print("-----")
# print("転置畳み込み")
# m = nn.ConvTranspose2d(1, 1, 2, stride=1, bias=False)
# m.weight.data[0, 0, 0, 0] = 1
# m.weight.data[0, 0, 0, 1] = 2
# m.weight.data[0, 0, 1, 0] = 3
# m.weight.data[0, 0, 1, 1] = 4
# print("カーネル")
# print(m.weight)
# print("出力")
# print(m(input))

In [ ]:
class Generator(nn.Module):

    def __init__(self, z_dim=20, image_size=64):
        super(Generator, self).__init__()

        self.layer1 = nn.Sequential(
            nn.ConvTranspose2d(z_dim, image_size * 8,
                               kernel_size=4, stride=1),
            nn.BatchNorm2d(image_size * 8),
            nn.ReLU(inplace=True))

        self.layer2 = nn.Sequential(
            nn.ConvTranspose2d(image_size * 8, image_size * 4,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(image_size * 4),
            nn.ReLU(inplace=True))

        self.layer3 = nn.Sequential(
            nn.ConvTranspose2d(image_size * 4, image_size * 2,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(image_size * 2),
            nn.ReLU(inplace=True))

        self.layer4 = nn.Sequential(
            nn.ConvTranspose2d(image_size * 2, image_size,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(image_size),
            nn.ReLU(inplace=True))

        self.last = nn.Sequential(
            nn.ConvTranspose2d(image_size, 1, kernel_size=4,
                               stride=2, padding=1),
            nn.Tanh())
        # 주의: 흑백 이미지이므로 출력 채널은 하나 뿐

    def forward(self, z):
        out = self.layer1(z)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.last(out)

        return out

In [ ]:
# 동작확인
import matplotlib.pyplot as plt
%matplotlib inline

G = Generator(z_dim=20, image_size=64)

# 난수 입력
input_z = torch.randn(1, 20)

# 텐서 크기 (1, 20, 1, 1)으로 변경
input_z = input_z.view(input_z.size(0), input_z.size(1), 1, 1)

# 가짜 이미지 출력
fake_images = G(input_z)

img_transformed = fake_images[0][0].detach().numpy()
plt.imshow(img_transformed, 'gray')
plt.show()

In [ ]:
class Discriminator(nn.Module):

    def __init__(self, z_dim=20, image_size=64):
        super(Discriminator, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, image_size, kernel_size=4,
                      stride=2, padding=1),
            nn.LeakyReLU(0.1, inplace=True))
        # 주의: 흑백 이미지이므로 입력 채널은 하나뿐

        self.layer2 = nn.Sequential(
            nn.Conv2d(image_size, image_size*2, kernel_size=4,
                      stride=2, padding=1),
            nn.LeakyReLU(0.1, inplace=True))

        self.layer3 = nn.Sequential(
            nn.Conv2d(image_size*2, image_size*4, kernel_size=4,
                      stride=2, padding=1),
            nn.LeakyReLU(0.1, inplace=True))

        self.layer4 = nn.Sequential(
            nn.Conv2d(image_size*4, image_size*8, kernel_size=4,
                      stride=2, padding=1),
            nn.LeakyReLU(0.1, inplace=True))

        self.last = nn.Conv2d(image_size*8, 1, kernel_size=4, stride=1)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.last(out)

        return out

In [ ]:
# 동작확인
D = Discriminator(z_dim=20, image_size=64)

# 가짜 이미지 생성
input_z = torch.randn(1, 20)
input_z = input_z.view(input_z.size(0), input_z.size(1), 1, 1)
fake_images = G(input_z)

# 가짜 이미지를 D에 입력
d_out = D(fake_images)

# 출력 d_out에 시그모이드를 곱하여 0에서 1로 변환
print(nn.Sigmoid()(d_out))

In [ ]:
# D 오차 함수의 이미지 구현
# maximize log(D(x)) + log(1 - D(G(z)))


# ※ x가 정의되지 않아 오류 발생
#---------------


# 정답 라벨 작성
mini_batch_size = 2
label_real = torch.full((mini_batch_size,), 1)

# 가짜 라벨 작성
label_fake = torch.full((mini_batch_size,), 0)

# 오차 함수 정의
criterion = nn.BCEWithLogitsLoss(reduction='mean')

# 진짜 이미지 판정
d_out_real = D(x)

# 가짜 이미지를 생성하여 판정
input_z = torch.randn(mini_batch_size, 20)
input_z = input_z.view(input_z.size(0), input_z.size(1), 1, 1)
fake_images = G(input_z)
d_out_fake = D(fake_images)

# 오차 계산
d_loss_real = criterion(d_out_real.view(-1), label_real)
d_loss_fake = criterion(d_out_fake.view(-1), label_fake)
d_loss = d_loss_real + d_loss_fake

In [ ]:
# G의 오차 함수 이미지 구현
# maximize log(D(G(z)))


# ※ x가 정의되지 않아 오류 발생
#---------------


# 가짜 이미지를 생성하여 판정
input_z = torch.randn(mini_batch_size, 20)
input_z = input_z.view(input_z.size(0), input_z.size(1), 1, 1)
fake_images = G(input_z)
d_out_fake = D(fake_images)

# 오차 계산
g_loss = criterion(d_out_fake.view(-1), label_real)

In [ ]:
def make_datapath_list():
    """학습 및 검증 이미지 데이터와 어노테이션 데이터의 파일 경로 리스트 작성 """

    train_img_list = list()  # 이미지 파일 경로 저장

    for img_idx in range(200):
        img_path = "./data/img_78/img_7_" + str(img_idx)+'.jpg'
        train_img_list.append(img_path)

        img_path = "./data/img_78/img_8_" + str(img_idx)+'.jpg'
        train_img_list.append(img_path)

    return train_img_list

In [ ]:
class ImageTransform():
    """이미지의 전처리 클래스"""

    def __init__(self, mean, std):
        self.data_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])

    def __call__(self, img):
        return self.data_transform(img)

In [ ]:
class GAN_Img_Dataset(data.Dataset):
    """이미지의 데이터셋 클래스. 파이토치의 데이터셋 클래스를 상속"""

    def __init__(self, file_list, transform):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        '''이미지 매수 반환'''
        return len(self.file_list)

    def __getitem__(self, index):
        '''전처리한 이미지의 텐서 형식 데이터 취득'''

        img_path = self.file_list[index]
        img = Image.open(img_path)  # [높이][폭]흑백

        # 이미지 전처리
        img_transformed = self.transform(img)

        return img_transformed

In [ ]:
# DataLoader 작성과 동작 확인

# 파일 리스트 작성
train_img_list=make_datapath_list()

# Dataset 작성
mean = (0.5,)
std = (0.5,)
train_dataset = GAN_Img_Dataset(
    file_list=train_img_list, transform=ImageTransform(mean, std))

# DataLoader 작성
batch_size = 64

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

# 동작 확인
batch_iterator = iter(train_dataloader)  # 반복자로 변환
imges = next(batch_iterator)  # 첫 번째 요소를 꺼낸다.
print(imges.size())  # torch.Size([64, 1, 64, 64])

In [ ]:
# 네트워크 초기화
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        # Conv2dとConvTranspose2d 초기화
        nn.init.normal_(m.weight.data, 0.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
    elif classname.find('BatchNorm') != -1:
        # BatchNorm2d 초기화
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)


# 초기화 실시
G.apply(weights_init)
D.apply(weights_init)

print("네트워크 초기화 완료")

In [ ]:
# 모델을 학습시키는 함수 작성

def train_model(G, D, dataloader, num_epochs):

    # GPU를 사용할 수 있는지 확인
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 장치：", device)

    # 최적화 기법 설정
    g_lr, d_lr = 0.0001, 0.0004
    beta1, beta2 = 0.0, 0.9
    g_optimizer = torch.optim.Adam(G.parameters(), g_lr, [beta1, beta2])
    d_optimizer = torch.optim.Adam(D.parameters(), d_lr, [beta1, beta2])

    # 오차함수 정의
    criterion = nn.BCEWithLogitsLoss(reduction='mean')

    # 파라미터를 하드코딩
    z_dim = 20
    mini_batch_size = 64

    # 네트워크를 GPU로
    G.to(device)
    D.to(device)

    G.train()  # 모델을 훈련모드로
    D.train()  # 모델을 훈련모드로

    # 네트워크가 어느 정도 고정되면 고속화시킨다.
    torch.backends.cudnn.benchmark = True

    # 이미지 매수
    num_train_imgs = len(dataloader.dataset)
    batch_size = dataloader.batch_size

    # 반복 카운터 설정
    iteration = 1
    logs = []

    # 에폭 루프
    for epoch in range(num_epochs):
        # 개시 시간 저장
        t_epoch_start = time.time()
        epoch_g_loss = 0.0  # 에폭의 손실 합
        epoch_d_loss = 0.0  # 에폭의 손실 합

        print('-------------')
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-------------')
        print('（train）')

        # 데이터 로더에서 미니 배치씩 꺼내는 루프
        for imges in dataloader:

            # --------------------
            # 1. Discriminator 학습
            # --------------------
            # 미니 배치 크기가 1이면 배치 정규화에서 오류가 발생하므로 피한다.
            if imges.size()[0] == 1:
                continue

            # GPU를 사용할 수 있다면 GPU로 데이터를 보낸다.
            imges = imges.to(device)

            # 정답 라벨과 가짜 라벨 작성
            # 에폭의 마지막 반복은 미니 배치 수가 줄어든다.
            mini_batch_size = imges.size()[0]
            label_real = torch.full((mini_batch_size,), 1).to(device)
            label_fake = torch.full((mini_batch_size,), 0).to(device)

            # 진짜 이미지 판정
            d_out_real = D(imges)

            # 가짜 이미지를 생성하여 판정
            input_z = torch.randn(mini_batch_size, z_dim).to(device)
            input_z = input_z.view(input_z.size(0), input_z.size(1), 1, 1)
            fake_images = G(input_z)
            d_out_fake = D(fake_images)

            # 오차 계산
            d_loss_real = criterion(d_out_real.view(-1), label_real)
            d_loss_fake = criterion(d_out_fake.view(-1), label_fake)
            d_loss = d_loss_real + d_loss_fake

            # 역전파
            g_optimizer.zero_grad()
            d_optimizer.zero_grad()

            d_loss.backward()
            d_optimizer.step()

            # --------------------
            # 2. Generator 학습
            # --------------------
            # 가짜 이미지를 생성하여 판정
            input_z = torch.randn(mini_batch_size, z_dim).to(device)
            input_z = input_z.view(input_z.size(0), input_z.size(1), 1, 1)
            fake_images = G(input_z)
            d_out_fake = D(fake_images)

            # 오차 계산
            g_loss = criterion(d_out_fake.view(-1), label_real)

            # 역전파
            g_optimizer.zero_grad()
            d_optimizer.zero_grad()
            g_loss.backward()
            g_optimizer.step()

            # --------------------
            # 3. 기록
            # --------------------
            epoch_d_loss += d_loss.item()
            epoch_g_loss += g_loss.item()
            iteration += 1

        # 에폭의 phase별 손실과 정답률
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_D_Loss:{:.4f} ||Epoch_G_Loss:{:.4f}'.format(
            epoch, epoch_d_loss/batch_size, epoch_g_loss/batch_size))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

    return G, D

In [ ]:
# 학습 및 검증 실행
# 6분 정도 걸림
num_epochs = 200
G_update, D_update = train_model(
    G, D, dataloader=train_dataloader, num_epochs=num_epochs)


In [ ]:
# 생성 이미지와 훈련 데이터 시각화
# 이 셀은 괜찮은 느낌의 이미지가 생성될 떄까지 재실행한다.

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 입력 난수 생성
batch_size = 8
z_dim = 20
fixed_z = torch.randn(batch_size, z_dim)
fixed_z = fixed_z.view(fixed_z.size(0), fixed_z.size(1), 1, 1)

# 이미지 생성
G_update.eval()
fake_images = G_update(fixed_z.to(device))

# 훈련 데이턴
batch_iterator = iter(train_dataloader)  # 반복자로 변환
imges = next(batch_iterator)  # 첫 번째 요소를 꺼낸다.


# 출력
fig = plt.figure(figsize=(15, 6))
for i in range(0, 5):
    # 상단에 훈련 데이터 표시
    plt.subplot(2, 5, i+1)
    plt.imshow(imges[i][0].cpu().detach().numpy(), 'gray')

    # 하단에 생성 데이터를 표시
    plt.subplot(2, 5, 5+i+1)
    plt.imshow(fake_images[i][0].cpu().detach().numpy(), 'gray')

In [ ]:
# Attentiom Map을 출력
fig = plt.figure(figsize=(15, 6))
for i in range(0, 5):

    # 상단에 생성한 이미지 데이터 표시
    plt.subplot(2, 5, i+1)
    plt.imshow(fake_images[i][0].cpu().detach().numpy(), 'gray')

    # 하단에 Attention Map 1 이미지
    plt.subplot(2, 5, 5+i+1)
    am = am1[i].view(16, 16, 16, 16)
    am = am[7][7]  # 중앙에 주목
    plt.imshow(am.cpu().detach().numpy(), 'Reds')
